In [1]:
## LOADING THE REQUIRED LIBRARIES

import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm 
import tensorflow as tf
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from  sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="3,4,6"

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib

Num GPUs Available:  3


In [4]:
# Loading Dataset
## LOADING THE PROCESSED DATASET
df= pd.read_csv("DATA/etoori_train.csv")
df.columns = ["enc_input","dec_input"] 
df["dec_output"] = df.dec_input
df

,enc_input,dec_input,dec_output
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप ...,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपन...,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपन...
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल ...,देश में हिन्दी को विस्थापित करने का षड़यंत्र च...,देश में हिन्दी को विस्थापित करने का षड़यंत्र च...
2,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...
3,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...
4,तब तक के लिए हमें विराम ले की अनुमति दीजिए ।,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।
...,...,...,...
139995,लेकिन फिर भी फिल्म को लोगों जित निगेटिव कमेंट्...,लेकिन फिर भी फिल्म को लोगों ने जितने निगेटिव क...,लेकिन फिर भी फिल्म को लोगों ने जितने निगेटिव क...
139996,बेटी से छेड़छाड़ के साथ वह कई बार दुराचार की क...,बेटी से छेड़छाड़ के साथ उसने कई बार दुराचार की...,बेटी से छेड़छाड़ के साथ उसने कई बार दुराचार की...
139997,आज यह कलाकार बालीवुड की चकाचौंध से दूर अपने गा...,आज यह कलाकार बालीवुड की चकाचौंध से दूर अपने गा...,आज यह कलाकार बालीवुड की चकाचौंध से दूर अपने गा...
139998,यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओं में सफ...,यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओं में सफ...,यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओं में सफ...


In [5]:
## THE INPUTS TO THE DECODER REQUIRES SPECIAL TOKENS FOR THE START AND THE END SO WE ARE GOING TO USE 
## <start> AS BEGINING TOKEN
## <end>  AS END TOKEN

df["dec_input"]= "<start> " + df["dec_input"]
df["dec_output"] =  df["dec_output"] + " <end>" 
df

,enc_input,dec_input,dec_output
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप ...,<start> परन्तु वे दोनों उन बातों को ज़्यादा सम...,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपन...
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल ...,<start> देश में हिन्दी को विस्थापित करने का षड...,देश में हिन्दी को विस्थापित करने का षड़यंत्र च...
2,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...,<start> तीन साल पहले कातिलाना हमले के प्रकरण म...,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...
3,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...,<start> रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवें...,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...
4,तब तक के लिए हमें विराम ले की अनुमति दीजिए ।,<start> तब तक के लिए हमें विराम लेने की अनुमति...,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।...
...,...,...,...
139995,लेकिन फिर भी फिल्म को लोगों जित निगेटिव कमेंट्...,<start> लेकिन फिर भी फिल्म को लोगों ने जितने न...,लेकिन फिर भी फिल्म को लोगों ने जितने निगेटिव क...
139996,बेटी से छेड़छाड़ के साथ वह कई बार दुराचार की क...,<start> बेटी से छेड़छाड़ के साथ उसने कई बार दु...,बेटी से छेड़छाड़ के साथ उसने कई बार दुराचार की...
139997,आज यह कलाकार बालीवुड की चकाचौंध से दूर अपने गा...,<start> आज यह कलाकार बालीवुड की चकाचौंध से दूर...,आज यह कलाकार बालीवुड की चकाचौंध से दूर अपने गा...
139998,यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओं में सफ...,<start> यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओ...,यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओं में सफ...


In [6]:
# Splitting And Sampling around 100k datapoints
#THE TOTAL DATASET HAS 500K DATAPOINTS WHICH WILL TAKE MUCH HIGHER TRAINING TIME. THEREFORE I AM SAMPLING ONE-FIFTH OF THE TOTAL DATASET

#df_sampled = pd.concat((df[df.enc_input].sample(frac= 0.2,random_state=1)))
#df_sampled = df.sample(frac = 0.4)
print(df.shape)
#print(df_sampled.shape)


(140000, 3)


In [7]:
## ONCE THE DATA IS SAMPLED WE ARE SPLITTIND THE DATA IN TO TRAIN AND TEST
df_train ,df_val = train_test_split(df,test_size=0.2,random_state = 3)

In [8]:
## IN THE COLUMN WHICH HAS DECODER INPUTS ADDING "<end>" TOKEN TO BE LEARNED BY THE TOKENIZER
df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"
df_train

,enc_input,dec_input,dec_output
1522,अवध के माटी रतनों को ढूंढने वाली इस संस्था और ...,<start> अवध के माटी रतनों को ढूंढने वाली इस सं...,अवध के माटी रतनों को ढूंढने वाली इस संस्था और ...
107460,यही तो कला के दिन है ।,<start> यही तो कला के दिन हैं ।,यही तो कला के दिन हैं । <end>
134546,इसीलिए तो जिधर सिर मुडाता हूँ ओले उसी तरफ पड़न...,<start> इसीलिए तो जिधर सिर मुडाता हूँ ओले उसी ...,इसीलिए तो जिधर सिर मुडाता हूँ ओले उसी तरफ पड़न...
120290,फोन पर बात बंद हुई तो सीआई रोते हुए बताया कि त...,<start> फोन पर बात बंद हुई तो सीआई ने रोते हुए...,फोन पर बात बंद हुई तो सीआई ने रोते हुए बताया क...
109794,परिजनों ने उसे थेने लाए ।,<start> परिजनों ने उसे थाने लाए ।,परिजनों ने उसे थाने लाए । <end>
...,...,...,...
11261,प्रायः झपट्टा मारती है ।,<start> प्रायः झपट्टा मारती हैं ।,प्रायः झपट्टा मारती हैं । <end>
104704,अप गांव को उन्नत कर का उनका एक दीर्घकालीन सपना...,<start> अपने गांव को उन्नत करने का उनका एक दीर...,अपने गांव को उन्नत करने का उनका एक दीर्घकालीन ...
48056,लेकिन कई क्षेत्र ऐसे हैंं जहां इस लाइन के बिछा...,<start> लेकिन कई क्षेत्र ऐसे हैं जहां इस लाइन ...,लेकिन कई क्षेत्र ऐसे हैं जहां इस लाइन के बिछा ...
77049,वास्तव में हज़रत अली अलैहिस्सलाम साहस और निर्भ...,<start> वास्तव में हज़रत अली अलैहिस्सलाम साहस ...,वास्तव में हज़रत अली अलैहिस्सलाम साहस और निर्भ...


In [9]:
## VALIDATION DATA
df_val

,enc_input,dec_input,dec_output
90522,महुआ का तेल जलाने के काम में लाया जाता हैं ।,<start> महुआ का तेल जलाने के काम में लाया जाता...,महुआ का तेल जलाने के काम में लाया जाता है । <...
133504,आजकल ट्रेन में एक से ज़्यादा टीटी भी होते हैंं...,<start> आजकल ट्रेन में एक से ज़्यादा टीटी भी ह...,आजकल ट्रेन में एक से ज़्यादा टीटी भी होते हैं ...
2373,शाम के वक्त बारी बारी से घर के सभी सदस्य रश्मि...,<start> शाम के वक्त बारी बारी से घर के सभी सदस...,शाम के वक्त बारी बारी से घर के सभी सदस्य रश्मि...
136466,उनमें शिवकुमार दीपक प्रमुख है ।,<start> उनमें शिवकुमार दीपक प्रमुख हैं ।,उनमें शिवकुमार दीपक प्रमुख हैं । <end>
5198,हमारे देश में संविधान द्वारा अब स्त्रियों को अ...,<start> हमारे देश में संविधान द्वारा अब स्त्रि...,हमारे देश में संविधान द्वारा अब स्त्रियों को अ...
...,...,...,...
111735,सरकार किसी पुल या सड़क का लोकार्पण करती हैं तो...,<start> सरकार किसी पुल या सड़क का लोकार्पण करत...,सरकार किसी पुल या सड़क का लोकार्पण करती है तो ...
19247,उन्होंने हमें सिखाया कि कैसे संघर्ष किया जाता ...,<start> उन्होंने हमें सिखाया कि कैसे संघर्ष कि...,उन्होंने हमें सिखाया कि कैसे संघर्ष किया जाता ...
57036,लेकिन यह इतिहास का अन्त नहीं हैं ।,<start> लेकिन यह इतिहास का अन्त नहीं है ।,लेकिन यह इतिहास का अन्त नहीं है । <end>
73689,रतन टाटा मल्टीब्रांड रिटेल में एफडीआई का स्वाग...,<start> रतन टाटा ने मल्टीब्रांड रिटेल में एफडी...,रतन टाटा ने मल्टीब्रांड रिटेल में एफडीआई का स्...


In [10]:
## HERE I AM SAMPLING 1000 POINTS FROM THE DATAFRAME AS TEST DATA WHICH ARE NOT PRESEENT IN THE TRAIN AND VALIDAION DATA
np.random.seed(5) 
df_test = df.loc[np.random.choice(np.array([x for x in df.index.values]),1000,replace= False,)]
df_test

,enc_input,dec_input,dec_output
80240,हमें आने वाले हफ्तों में रोवियो मोबाइल द्वारा ...,<start> हमें आने वाले हफ्तों में रोवियो मोबाइल...,हमें आने वाले हफ्तों में रोवियो मोबाइल द्वारा ...
32433,आज कमेन्ट मोदेरेशन सक्षम हैंं ।,<start> आज कमेन्ट मोदेरेशन सक्षम हैं ।,आज कमेन्ट मोदेरेशन सक्षम हैं । <end>
135530,राजस् थेन स्कूल टीचर्स यूनियन के अध्यक्ष आर पी...,<start> राजस् थान स्कूल टीचर्स यूनियन के अध्यक...,राजस् थान स्कूल टीचर्स यूनियन के अध्यक्ष आर पी...
3354,सेक्स का कारक ग्रह भी शुक्र ही हैं तथा शुक्र ए...,<start> सेक्स का कारक ग्रह भी शुक्र ही है तथा ...,सेक्स का कारक ग्रह भी शुक्र ही है तथा शुक्र एक...
125119,वह एक गिलास शर्बत,<start> उसने एक गिलास शर्बत,उसने एक गिलास शर्बत <end>
...,...,...,...
123821,बीकॉम ऑनर्स और इको ऑनर्स कोर्स स्टूडेंट की पसं...,<start> बीकॉम ऑनर्स और इको ऑनर्स कोर्स स्टूडें...,बीकॉम ऑनर्स और इको ऑनर्स कोर्स स्टूडेंट की पसं...
40525,मरने वालों में बस में सवार यात्री थी ।,<start> मरने वालों में बस में सवार यात्री थे ।,मरने वालों में बस में सवार यात्री थे । <end>
72286,यान्त्रिक रूप से क्षमा मांगना सम्पर्क बढ़ा का ...,<start> यान्त्रिक रूप से क्षमा मांगना सम्पर्क ...,यान्त्रिक रूप से क्षमा मांगना सम्पर्क बढ़ाने क...
90240,मैंनेने इस शहर में खुद का स् वागत बांहें खोलकर...,<start> मैंने इस शहर में खुद का स् वागत बांहें...,मैंने इस शहर में खुद का स् वागत बांहें खोलकर न...


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
## TOKENIZER FOR ENCODER INPUT
tk_inp = Tokenizer()
tk_inp.fit_on_texts(df_train.enc_input.apply(str))

In [13]:
# TOKENIZER FOR DECODER INPUT
tk_out = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' )
tk_out.fit_on_texts(df_train.dec_input.apply(str))

In [14]:
## THIS CLASS CONVERTS TEXT DATA TO INTEGER SEQUENCES AND RETURNS THE PADDED SEQUENCES
class Dataset :
    def __init__(self, data , tk_inp ,tk_out, max_len):
        ## SETTING THE REQUIRED ATTRIBUTES
        self.encoder_inp = data["enc_input"].apply(str).values
        self.decoder_inp = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp
        self.tk_out = tk_out
        self.max_len = max_len
        
    def __getitem__(self,i):
        # INPUT SEQUENCES
        self.encoder_seq = self.tk_inp.texts_to_sequences([self.encoder_inp[i]])
        # DECODER INPUT SEQUENCES 
        self.decoder_inp_seq = self.tk_out.texts_to_sequences([self.decoder_inp[i]])
        # DECODER INPUT SEQUENCES
        self.decoder_out_seq = self.tk_out.texts_to_sequences([self.decoder_out[i]])
        
        # PADDING THE ENCODER INPUT SEQUENCES
        self.encoder_seq = pad_sequences(self.encoder_seq, padding="post",maxlen = self.max_len)
        # PADDING THE DECODER INPUT SEQUENCES
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, padding="post",maxlen = self.max_len)
        # PADDING DECODER OUTPUT SEQUENCES
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq ,padding="post", maxlen = self.max_len)
        return self.encoder_seq ,  self.decoder_inp_seq,  self.decoder_out_seq
    
    def __len__(self):
        # RETURN THE LEN OF INPUT ENDODER
        return len(self.encoder_inp)

In [15]:
## THIS CLASS CONVERTES THE DATASET INTO THE REQUIRED BATCH SIZE

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self,batch_size,dataset):
        # INTIALIZING THE REQUIRED VARIABLES 
        self.dataset = dataset
        self.batch_size = batch_size
        self.totl_points = self.dataset.encoder_inp.shape[0]
        
    def __getitem__(self,i):
        # STATING THE START AND STOP VATIABLE CONTAINGING INDEX VALUES FOR EACH BATCH
        start = i * self.batch_size
        stop = (i+1)*self.batch_size
        
        # PLACEHOLDERS FOR BATCHED DATA
        batch_enc =[]
        batch_dec_input = []
        batch_dec_out =[]

        for j in range(start,stop): 
            
            a,b,c = self.dataset[j] 
            batch_enc.append(a[0]) 
            batch_dec_input.append(b[0])
            batch_dec_out.append(c[0]) 
        
        # Conveting list to array   
        batch_enc = (np.array(batch_enc)) 
        batch_dec_input = np.array(batch_dec_input)
        batch_dec_out = np.array(batch_dec_out)
        
        return [batch_enc , batch_dec_input],batch_dec_out
    
    def __len__(self):
        # Returning the number of batches
        return int(self.totl_points/self.batch_size)

In [17]:
# FORMING OBJECTS OF DATASET AND DATALOADER FOR TRAIN DATASET
train_dataset = Dataset(df_train,tk_inp,tk_out,51)
train_dataloader = Dataloader( batch_size = 32, dataset=train_dataset)

# FORMING OBJECTS OF DATASET AND DATALOADER FOR VALIDATION DATASET
val_dataset = Dataset(df_val , tk_inp,tk_out,51)
val_dataloader = Dataloader(batch_size=32 , dataset=val_dataset)

In [18]:
#Attention Model
from tensorflow.keras import layers
from tensorflow.keras import Model

In [19]:
## DEFINING THE ENCODER LAYER AS A FUNCTION

class Encoder(tf.keras.layers.Layer):


    
    def __init__(self, vocab_size,emb_dims, enc_units, input_length,batch_size):
        super().__init__()
        # INITIALIZING THE REQUIRED VARIABLES
        self.batch_size=batch_size # BATHCH SIZE
        self.enc_units = enc_units # ENCODER UNITS

        # EMBEDDING LAYER
        self.embedding= layers.Embedding(vocab_size ,emb_dims) 
        # LSTM LAYER WITH RETURN SEQ AND RETURN STATES
        self.lstm = layers.LSTM(self.enc_units,return_state= True,return_sequences =  True) 
    def call(self, enc_input , states):
      
        # FORMING THE EMBEDDED VECTOR 
        emb = self.embedding(enc_input)
        # PASSING THE EMBEDDED VECTIO THROUGH LSTM LAYERS 
        enc_output,state_h,state_c = self.lstm(emb,initial_state=states)
        #RETURNING THE OUTPUT OF LSTM LAYER
        return enc_output,state_h,state_c 
    def initialize(self,batch_size):

        return tf.zeros(shape=(batch_size,self.enc_units)),tf.zeros(shape=(batch_size,self.enc_units))

In [20]:
# THIS IS ATTNETION LAYER FOR DOT MODEL
class Attention(tf.keras.layers.Layer):
    
    def __init__(self,units):
        super().__init__()
        # INITIALIZING THE DENSE LAYER W1
        self.W1 = layers.Dense(units)
        # INITIALIZING THE DENSE LAYER W2
        self.W2 = layers.Dense(units)
        # INITIALIZING THE DENSE LAYER V
        self.v = layers.Dense(1)
        
    def call(self,enc_output,dec_state):
        # EXPANDING THE DIMENSION OF DECODER STATE  EG. FROM (16,32) TO (16,32,1)
        dec_state =  tf.expand_dims(dec_state,axis=1)
        
        # FINDING THE SCORE FOR CONCAT MODEL
        score = self.v(tf.nn.tanh(
            self.W1(dec_state)+ self.W2(enc_output)
        ))
        # APPLYING SOFTMAX TO THE AXIS 1
        # OUPUT SHAPE = (16,13,1)
        att_weights = tf.nn.softmax(score,axis=1)
        
        # CALCULATING THE CONTEXT VECTOR BY FIRST ELEMENTWISE MULTIPLICATION AND THEN ADDING THE AXIS 1
        # (16,13,1)*(16,13,32)=(16,13,32)
        context_vec  = att_weights* enc_output
        
        # (16,13,32) SUM AND REDUCE THE DIMENSION AT AXIS 1 => (16,32)
        context_vec = tf.reduce_sum(context_vec,axis=1)
        
        # RETURNING THE CONTEXT VECTOR AND ATTENTION WEIGHTS
        return context_vec,att_weights

In [21]:
class Onestepdecoder(tf.keras.Model):
    '''THIS MODEL OUTPUTS THE RESULT OF DECODER FOR ONE TIME SETP GIVEN THE INPUT FOR PRECIOVE TIME STEP'''
    
    def __init__(self, vocab_size,emb_dims, dec_units, input_len,att_units,batch_size):
        super().__init__()
        # INTITALIZING THE REQUIRED VARIABLES
        # EMBEDDING LAYERS
        self.emb = layers.Embedding(vocab_size,emb_dims,input_length= input_len)
        # ATTENTION LAYER
        self.att = Attention(att_units)
        # LSTM LAYER
        self.lstm = layers.LSTM(dec_units,return_sequences=True,return_state=True)
        # DENSE LAYER
        self.dense = layers.Dense(vocab_size,activation="softmax")

    def call(self, encoder_output , input , state_h,state_c):
        # FORMING THE EMBEDDED VECTOR FOR THE WORD
        # (32,1)=>(32,1,12)
        emb = self.emb(input)

        dec_output,dec_state_h,dec_state_c = self.lstm(emb, initial_state = [state_h,state_c] )

        # GETTING THE CONTEXT VECTOR AND ATTENTION WEIGHTS BASED ON THE ENCODER OUTPUT AND  DECODER STATE_H
        context_vec,alphas = self.att(encoder_output,dec_state_h)
        
        # CONCATINATING THE CONTEXT VECTOR(BY EXPANDING DIMENSION) AND ENBEDDED VECTOR
        dense_input =  tf.concat([tf.expand_dims(context_vec,1),dec_output],axis=-1)
        
        # PASSING THE DECODER OUTPUT THROUGH DENSE LAYER WITH UNITS EQUAL TO VOCAB SIZE
        fc = self.dense(dense_input)
        
        # RETURNING THE OUTPUT
        return fc , dec_state_h , dec_state_c , alphas

In [22]:
class Decoder(tf.keras.Model):
    '''THIS MODEL PERFORMS THE WHOLE DECODER OPERATION FOR THE COMPLETE SENTENCE'''
    def __init__(self, vocab_size,emb_dims, dec_units, input_len,att_units,batch_size):
        super().__init__()
        # INITIALIZING THE VARIABLES
        # LENGTH OF INPUT SENTENCE
        self.input_len = input_len
        # ONE STEP DECODER
        self.onestepdecoder = Onestepdecoder(vocab_size,emb_dims, dec_units, input_len,att_units,batch_size)

    def call(self,dec_input,enc_output,state_h,state_c):
        # THIS VATIABLE STORES THE VALUE OF STATE_H FOR THE PREVIOUS STATE
        current_state_h = state_h 
        current_state_c = state_c
        # THIS STORES THE DECODER OUTPUT FOR EACH TIME STEP
        pred = []
        # THIS STORED THE ALPHA VALUES
        alpha_values = []
        # FOR EACH WORD IN THE INPUT SENTENCE
        for i in range(self.input_len):
            
            # CURRENT WORD TO INPUT TO ONE STEP DECODER
            current_vec = dec_input[:,i]

            # EXPANDING THE DIMENSION FOR THE WORD
            current_vec = tf.expand_dims(current_vec,axis=-1)

            # PERFORMING THE ONE STEP DECODER OPERATION 
            dec_output,dec_state_h,dec_state_c,alphas = self.onestepdecoder(enc_output ,current_vec,current_state_h,current_state_c)

            #UPDATING THE CURRENT STATE_H
            current_state_h = dec_state_h
            current_state_c = dec_state_c

            #APPENDING THE DECODER OUTPUT TO "pred" LIST
            pred.append(dec_output)

            # APPENDING THE ALPHA VALUES
            alpha_values.append(alphas)
            
        # CONCATINATING ALL THE VALUES IN THE LIST
        output = tf.concat(pred,axis=1)
        # CONCATINATING ALL THE ALPHA VALUES IN THE LIST
        alpha_values = tf.concat(alpha_values,axis = -1)
        # RETURNING THE OUTPUT
        return output , alpha_values

In [23]:
class encoder_decoder(tf.keras.Model):
    '''THIS MODEL COMBINES ALL THE LAYERS AND FORM IN ENCODER DECODER MODEL WITH ATTENTION MECHANISM'''
    def __init__(self,enc_vocab_size,enc_emb_dim,enc_units,enc_input_length,
             dec_vocab_size,dec_emb_dim,dec_units,dec_input_length ,att_units, batch_size):
        # INITAILIZING ALL VARIABLES
        super().__init__()
        # BATCH SIZE
        self.batch_size = batch_size
        # INITIALIZING ENCODER LAYER
        self.encoder = Encoder(enc_vocab_size, enc_emb_dim,enc_units, enc_input_length,batch_size)
        # INITALIZING DECODER LAYER
        self.decoder = Decoder(dec_vocab_size ,dec_emb_dim,dec_units,dec_input_length  ,att_units, batch_size)

    def call(self,data):
        # THE INPUT OF DATALOADER IS IN A LIST FORM FOR EACH BATCH IT GIVER TWO INPUTS
        # INPUT1 IS FOR ENCODER
        # INPUT2 IS FOR DECODER
        inp1 , inp2 = data
        # PASSING THE INPUT1 TO ENCODER LAYER
        enc_output, enc_state_h, enc_state_c = self.encoder(inp1,self.encoder.initialize(self.batch_size))
        # PASSING INPUT2 TO THE DECODER LAYER
        dec_output , alphas = self.decoder(inp2 , enc_output,enc_state_h,enc_state_c)
        # THE OUTPUT OF MODEL IS ONLY DECODER OUTPUT THE ALPHA VALUES ARE IGNORED HERE
        return dec_output

In [25]:
# INITAILZING THE MODEL
model = encoder_decoder(enc_vocab_size=len(tk_inp.word_index)+1,
                         enc_emb_dim = 300,
                         enc_units=256,enc_input_length=51,
                         dec_vocab_size =len(tk_out.word_index)+1,
                         dec_emb_dim =300,
                         dec_units=256,
                         dec_input_length = 51,
                         att_units=256,
                         batch_size=32)

In [26]:
callback =[ tf.keras.callbacks.ModelCheckpoint( "models/attention_concat_best.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001),
            tf.keras.callbacks.TensorBoard("models/attention_concat_logs_save",histogram_freq=1)
]

train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()

model.compile(optimizer="adam",loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(train_dataloader, steps_per_epoch=train_steps,epochs= 60,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback)

2022-10-19 00:16:12.004043: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101


Epoch 1/60
3500/3500 [==============================] - 1504s 405ms/step - loss: 1.4748 - accuracy: 0.8135 - val_loss: 0.4440 - val_accuracy: 0.9466
Epoch 2/60
3500/3500 [==============================] - 1400s 400ms/step - loss: 0.3130 - accuracy: 0.9626 - val_loss: 0.1835 - val_accuracy: 0.9762
Epoch 3/60
3500/3500 [==============================] - 1400s 400ms/step - loss: 0.1297 - accuracy: 0.9816 - val_loss: 0.1181 - val_accuracy: 0.9839
Epoch 4/60
3500/3500 [==============================] - 1399s 400ms/step - loss: 0.0569 - accuracy: 0.9910 - val_loss: 0.0961 - val_accuracy: 0.9862
Epoch 5/60
3500/3500 [==============================] - 1399s 400ms/step - loss: 0.0300 - accuracy: 0.9954 - val_loss: 0.0876 - val_accuracy: 0.9872
Epoch 6/60
3500/3500 [==============================] - 1399s 400ms/step - loss: 0.0215 - accuracy: 0.9968 - val_loss: 0.0815 - val_accuracy: 0.9879
Epoch 7/60
3500/3500 [==============================] - 1400s 400ms/step - loss: 0.0173 - accuracy: 0.9977

In [68]:
#model.build([(512,35),(512,35)])
#(batch_size,maxlen)
model.build([(32,51),(32,51)])
model.summary()

Model: "encoder_decoder_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  22719068  
                                                                 
 decoder_1 (Decoder)         multiple                  60027632  
                                                                 
Total params: 82,746,700
Trainable params: 82,746,700
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.load_weights("models/attention_concat_best.h5")

In [29]:
def predict(ita_text,model):
    '''THIS FUNCTION IS USED IN INFERENCE TIME WHICH GIVEN ANY SENTENCE IN ITALIAN OUTPUTS THE ENGLISH SENTENCE AND ALPHA VALUES'''
    # FORMING TOKENIZED SEQUENCES FOR INPUT SENTENCE
    seq = tk_inp.texts_to_sequences([ita_text])
    # PADDING THE SEQUENCES
    seq = pad_sequences(seq,maxlen = 20 , padding="post")
    # INITIALIZING THE STATES FOR INPUTING TO ENCODER
    state = model.layers[0].initialize(1)
    # GETTING THE ENCODED OUTPUT
    enc_output,state_h,state_c= model.layers[0](seq,state)
    # VARIABLE TO STORE PREDICTED SENTENCE
    pred = []
    # THIS VARIABLE STORES THE STATE TO BE INPUTED TO ONE STEP ENCODER
    input_state_h = state_h
    input_state_c = state_c
    # THIS VARIABLE STORES THE VECTOR TO VE INPUTED TO ONE STEP ENCODER
    current_vec = tf.ones((1,1))
    # THIS VARIABLE WILL STORE ALL THE ALPHA VALUES OUTPUTS
    alpha_values = []

    for i in range(20):
        # PASSING THE REQUIRED VARIABLE TO ONE STEP ENCODER LAYER
        fc , dec_state_h ,dec_state_c, alphas = model.layers[1].layers[0](enc_output , current_vec ,input_state_h ,input_state_c)
        #APPENDING THE ALPHA VALUES TO THE LIST "alpha_values"
        alpha_values.append(alphas)
         # UPDATING THE CURRENT VECTOR 
        current_vec = np.argmax(fc , axis = -1)
         # UPDATING THE INPUT STATE
        input_state_h = dec_state_h
        input_state_c = dec_state_c
        # GETTING THE ACTUAL WORDS FRO THE TOKENIZED INDEXES
        pred.append(tk_out.index_word[current_vec[0][0]])
        # IF THE WORD "<end>" COMES THE LOOP WILL BREAK
        if tk_out.index_word[current_vec[0][0]]=="<end>":
              break
    # JOINING THE PREDICTED WORDS
    pred_sent = " ".join(pred)
    # CONCATINATING ALL THE ALPHA VALUES
    alpha_values = tf.squeeze(tf.concat(alpha_values,axis=-1),axis=0)
    # RETURNING THE PREDICTED SENTENCE AND ALPHA VALUES
    return  pred_sent , alpha_values

In [30]:
import nltk.translate.bleu_score as bleu

In [31]:
BLEU = []
index = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model)[0].split()
        act = [str(i.dec_output).split()]
        b = bleu.sentence_bleu(act,pred)
        BLEU.append(b)
    except:
        index.append(ind)
        continue
print("BELU = ", np.mean(BLEU))

2000it [04:44,  7.03it/s]

BELU =  0.7554539743100016


In [32]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[19],model)[0])
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[19])

INPUT SENTENCE ===>  वह जमाना गया जब धन कमाने में काफी समय लगता थे और कड़ी मेहनत करनी पड़ती थी ।
PREDICTED SENTENCE ===>  वह जमाना गया जब धन कमाने में काफी समय लगता था और कड़ी मेहनत करनी पड़ती थी । <end>
ACTUAL SENTENCE ===>  वह जमाना गया जब धन कमाने में काफी समय लगता था और कड़ी मेहनत करनी पड़ती थी ।  <end>


In [33]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[50],model)[0])
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[50])

INPUT SENTENCE ===>  नारों की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे बीन की धुन सांप को बाहर खीच लाती हैं ।
PREDICTED SENTENCE ===>  की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे बीन की धुन सांप को बाहर खीच लाती है ।
ACTUAL SENTENCE ===>  नारों की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे बीन की धुन सांप को बाहर खीच लाती है ।  <end>


In [34]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[4])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[4],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[4])

INPUT SENTENCE ===>  वह एक गिलास शर्बत
PREDICTED SENTENCE ===>  ('उसने एक गिलास शर्बत <end>', <tf.Tensor: shape=(20, 5), dtype=float32, numpy=
array([[9.99980330e-01, 3.38565878e-05, 3.34737149e-09, 1.54150968e-08,
        3.54617447e-09],
       [5.36854759e-06, 9.96379793e-01, 2.16728786e-05, 5.31993351e-07,
        1.10517124e-06],
       [2.66608311e-07, 3.56699037e-03, 9.99834538e-01, 1.25003382e-04,
        1.36153421e-05],
       [2.50011482e-07, 1.46757884e-05, 1.42085613e-04, 9.99401689e-01,
        6.00945379e-04],
       [2.13182346e-07, 2.19969661e-06, 1.65240044e-06, 4.68088809e-04,
        9.97370362e-01],
       [3.65056856e-08, 2.41503699e-06, 2.28207444e-08, 4.59519060e-06,
        2.00175145e-03],
       [1.10917696e-07, 1.69844967e-07, 7.72128528e-10, 5.44818306e-08,
        1.08415861e-05],
       [1.77963798e-07, 4.35481073e-09, 3.91599357e-11, 7.95768607e-09,
        1.22674066e-06],
       [3.57622048e-06, 7.50106366e-10, 6.77879541e-12, 6.59068577e-10,
        4

In [35]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[20])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[20],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[20])

INPUT SENTENCE ===>  गुजरात और यूपी भी में हुए ऐसे मामले
PREDICTED SENTENCE ===>  ('गुजरात और यूपी में भी हुए ऐसे मामले <end>', <tf.Tensor: shape=(20, 9), dtype=float32, numpy=
array([[9.98853922e-01, 3.24899411e-05, 1.24110766e-08, 6.57438237e-09,
        1.43162726e-08, 4.03919076e-09, 4.15274526e-09, 3.89571087e-09,
        3.35853144e-07],
       [8.63616006e-05, 9.99728620e-01, 1.60602099e-06, 4.72136236e-07,
        9.05473280e-06, 2.34476005e-07, 2.28071038e-08, 1.53203814e-07,
        1.05292065e-05],
       [3.40116203e-05, 1.93202126e-04, 9.99943852e-01, 1.02320351e-04,
        4.15916962e-04, 1.57351235e-06, 5.02141631e-07, 1.36452240e-06,
        2.14738952e-06],
       [1.56725764e-05, 2.01702496e-05, 7.31064256e-06, 9.99598444e-01,
        1.30946434e-03, 3.41392797e-06, 1.20849086e-06, 4.17459796e-06,
        1.74643228e-05],
       [2.84863418e-05, 4.97813380e-06, 3.47487439e-05, 1.88356353e-05,
        9.92558360e-01, 1.51248113e-03, 1.81295163e-05, 9.45820866e-05,
   

In [36]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[500])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[500],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[500])

INPUT SENTENCE ===>  अमेरिका भी इससे अलग नहीं हैं ।
PREDICTED SENTENCE ===>  ('अमेरिका भी इससे अलग नहीं है । <end>', <tf.Tensor: shape=(20, 8), dtype=float32, numpy=
array([[9.98489618e-01, 4.81580995e-04, 2.19598448e-07, 1.35816949e-08,
        1.94864236e-09, 9.12403131e-09, 1.03830771e-05, 1.54007648e-04],
       [2.72520178e-04, 9.98108029e-01, 1.19585848e-05, 1.40193407e-07,
        6.12408769e-07, 2.49883129e-07, 8.15184649e-06, 1.19040417e-03],
       [4.28637723e-05, 1.07149221e-03, 9.79689360e-01, 5.15208194e-05,
        6.34142907e-06, 4.35848187e-06, 3.04026571e-06, 8.49192031e-04],
       [1.60442651e-05, 1.49708620e-04, 2.02006809e-02, 9.98209715e-01,
        4.32106387e-03, 9.83662358e-06, 6.99895918e-06, 3.86773696e-04],
       [1.27743651e-05, 1.55754969e-05, 5.35609906e-05, 1.72620045e-03,
        9.95338202e-01, 1.28763886e-02, 1.24779399e-04, 1.46938735e-04],
       [2.33619521e-05, 1.30977842e-05, 3.63253312e-05, 1.16868978e-05,
        3.04110232e-04, 9.87013876e-0

In [37]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[700])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[700],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[700])

INPUT SENTENCE ===>  जनपथ के एक दुकानदार नितिन कहते है कि अबतक उन्होंने किसी को भी पॉलिथीन चेकिंग करते नहीं देखा
PREDICTED SENTENCE ===>  ('जनपथ ने अबतक एक एक दुकानदार नितिन कहते हैं कि अबतक उन्होंने किसी को भी पॉलिथीन चेकिंग करते नहीं देखा', <tf.Tensor: shape=(20, 20), dtype=float32, numpy=
array([[9.99808013e-01, 5.50184548e-01, 1.20955880e-03, 1.68752376e-04,
        9.25014228e-06, 9.27644635e-07, 1.59587131e-07, 5.04325826e-08,
        8.28998566e-07, 3.31519027e-06, 2.40469417e-05, 7.89095168e-07,
        3.59310860e-07, 7.36679056e-07, 1.18249838e-04, 1.24103494e-07,
        8.88263063e-09, 9.07024571e-07, 1.14412487e-05, 1.77769402e-06],
       [9.93301455e-07, 3.91819447e-01, 1.12764470e-01, 2.20605335e-03,
        2.05609304e-05, 6.29920905e-05, 2.19002268e-05, 4.12795771e-06,
        7.08638254e-06, 5.15242718e-05, 1.85320932e-05, 2.46812601e-06,
        1.52841380e-06, 9.12627627e-08, 1.00181273e-06, 9.44384038e-09,
        3.41153950e-09, 2.69732308e-08, 2.77529875e-06, 1.

In [38]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[900])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[900],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[900])

INPUT SENTENCE ===>  इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक दवाई बनाई हैं ।
PREDICTED SENTENCE ===>  ('इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक दवाई बनाई है । <end>', <tf.Tensor: shape=(20, 11), dtype=float32, numpy=
array([[9.9877590e-01, 1.3056111e-04, 8.2218043e-07, 1.0026201e-09,
        4.5531102e-08, 9.6673254e-08, 7.8770839e-08, 2.0109051e-07,
        3.5331817e-07, 8.0025274e-07, 2.8712081e-04],
       [2.4995950e-04, 9.9959677e-01, 5.1303894e-05, 3.2703622e-06,
        1.8026620e-06, 6.9205424e-07, 3.3638524e-04, 1.1284838e-05,
        9.7382963e-07, 2.4866944e-07, 1.0800510e-05],
       [1.0347508e-05, 5.3292606e-05, 7.1994048e-01, 8.9224667e-04,
        7.0513190e-05, 2.0528291e-06, 5.9535782e-06, 3.3524342e-05,
        4.0889546e-07, 4.1862837e-08, 1.4747013e-05],
       [2.4307187e-04, 5.0435851e-06, 2.7255586e-01, 9.9904162e-01,
        4.1560413e-04, 1.2742775e-05, 1.8226605e-05, 3.0321648e-04,
        1.4736061e-06, 9.3462899e-07, 1.2576739e-04],
       [1.9582672e-05, 

In [39]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[550])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[550],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[550])

INPUT SENTENCE ===>  पांचवें दिन फौज के लोगों को फकीरी दी जाती थे ।
PREDICTED SENTENCE ===>  ('पांचवें दिन फौज के लोगों को फकीरी दी जाती थी । <end>', <tf.Tensor: shape=(20, 12), dtype=float32, numpy=
array([[9.99896288e-01, 2.89322305e-02, 1.31963091e-07, 1.18955290e-08,
        8.82760673e-08, 9.17517582e-08, 1.66982339e-08, 1.55641061e-07,
        1.24816868e-08, 2.27507272e-07, 4.72933334e-06, 8.51445634e-08],
       [2.44058742e-06, 9.68366563e-01, 1.40338307e-04, 1.21325320e-06,
        1.14710092e-05, 8.89279590e-07, 7.31612658e-07, 1.72580951e-06,
        1.30168814e-06, 2.31445188e-06, 3.03350339e-06, 5.63738604e-06],
       [1.65161703e-06, 1.38122356e-03, 9.99459922e-01, 1.84743118e-03,
        2.77654590e-06, 3.29397080e-05, 1.56429189e-06, 9.67641517e-06,
        1.04639139e-05, 2.71209205e-06, 1.92263724e-06, 1.47236403e-06],
       [3.25636847e-06, 8.44085025e-06, 1.33890018e-04, 9.97226298e-01,
        8.09919220e-06, 1.75741866e-06, 4.27593841e-06, 1.86609923e-05,
     

In [40]:
%%time
predict(df_test.enc_input.values[50],model)[0]

CPU times: user 199 ms, sys: 0 ns, total: 199 ms
Wall time: 186 ms


'की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे बीन की धुन सांप को बाहर खीच लाती है ।'

In [41]:
def beam_search(input,model,k):
    seq = tk_inp.texts_to_sequences([input])
    seq = pad_sequences(seq,maxlen = 51,padding="post")

    state = model.layers[0].initialize(1)
    # GETTING THE ENCODED OUTPUT
    enc_output,enc_state_h,enc_state_c = model.layers[0](seq,state)
    

    input_state_h = enc_state_h
    input_state_c = enc_state_c 
    k_beams = [[tf.ones((1,1),dtype=tf.int32),0.0]]
    for i in range(51):
        candidates = []
        for sent_pred , prob in k_beams :
            if tk_out.word_index["<end>"] in sent_pred.numpy() :
                candidates += [[sent_pred , prob]]
            else:
               
                dec_input = model.layers[1].layers[0].layers[0](sent_pred)
                dec_output , dec_state_h , dec_state_c   =  model.layers[1].layers[0].layers[2](dec_input ,  initial_state =  [input_state_h , input_state_c])

                context_vec , alphas =  model.layers[1].layers[0].layers[1](enc_output,dec_state_h)

                # CONCATINATING THE CONTEXT VECTOR(BY EXPANDING DIMENSION) AND ENBEDDED VECTOR
                dense_input =  tf.concat([tf.expand_dims(context_vec,1),tf.expand_dims(dec_state_h,1)],axis=-1)
                
                # PASSING THE DECODER OUTPUT THROUGH DENSE LAYER WITH UNITS EQUAL TO VOCAB SIZE
                dense = model.layers[1].layers[0].layers[3](dense_input)

                pred = tf.argsort(dense, direction= 'DESCENDING')[:,:,:k]
                for w in range(k):
                  candidates += [[tf.concat((sent_pred, pred[:,:,w]) , axis=-1) , (prob + tf.math.log(dense[:,:,pred[:,:,w][0][0]])[0][0])]  ]
        k_beams = sorted(candidates,key=lambda tup:tup[1],reverse=True)[:k]

    all_sent = []
    for i,score in k_beams:
        sent = ""
        for j in range(1,51):
            sent +=  tk_out.index_word[i.numpy()[:,j][0]] +  " " 
            if tk_out.index_word[i.numpy()[:,j][0]] =="<end>":
                break
        all_sent.append((sent.strip(),score.numpy()))
    return all_sent

In [42]:
# VALIDATION BELU SCORE
BLEU_beam = []
index = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = beam_search(str(i.enc_input),model,3)[0][0].split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_beam.append(b)
    except:
        index.append(ind)
        continue

print("BELU Score = ",np.mean(BLEU_beam)) 

2000it [27:19,  1.22it/s]

BELU Score =  0.8733800699625884


In [43]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("="*50)
print("ACTUAL OUTPUT ===> ",df_test.dec_output.values[19])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[19],model,3))
for i in bm:
    print(i)

INPUT SENTENCE ===>  वह जमाना गया जब धन कमाने में काफी समय लगता थे और कड़ी मेहनत करनी पड़ती थी ।
ACTUAL OUTPUT ===>  वह जमाना गया जब धन कमाने में काफी समय लगता था और कड़ी मेहनत करनी पड़ती थी ।  <end>
BEAM SEARCH OUTPUT ,  SCORE
('वह जमाना गया जब धन कमाने में काफी समय लगता था और कड़ी मेहनत करनी पड़ती थी । <end>', -0.02508776)
('वह जमाना था जब धन कमाने में काफी समय लगता था और कड़ी मेहनत करनी पड़ती थी । <end>', -5.4596124)
('वह जमाना गया था धन कमाने में काफी समय लगता था और कड़ी मेहनत करनी पड़ती थी । <end>', -5.986592)


In [44]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("="*50)
print("ACTUAL OUTPUT ===> ",df_test.dec_output.values[50])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[50],model,3))
for i in bm:
    print(i)

INPUT SENTENCE ===>  नारों की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे बीन की धुन सांप को बाहर खीच लाती हैं ।
ACTUAL OUTPUT ===>  नारों की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे बीन की धुन सांप को बाहर खीच लाती है ।  <end>
BEAM SEARCH OUTPUT ,  SCORE
('नारों की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे बीन की धुन सांप को बाहर खीच लाती है । <end>', -0.090010874)
('नारों की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे आप की धुन सांप को बाहर खीच लाती है । <end>', -5.01867)
('नारों की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे सब बीन की धुन सांप को बाहर खीच लाती है । <end>', -5.509224)


In [45]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[500])
print("="*50)
print("ACTUAL OUTPUT ===> ",df_test.dec_output.values[500])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[500],model,3))
for i in bm:
    print(i)

INPUT SENTENCE ===>  अमेरिका भी इससे अलग नहीं हैं ।
ACTUAL OUTPUT ===>  अमेरिका भी इससे अलग नहीं है ।  <end>
BEAM SEARCH OUTPUT ,  SCORE
('अमेरिका भी इससे अलग नहीं है । <end>', -0.001691622)
('अमेरिका को इससे अलग नहीं है । <end>', -7.1688113)
('अमेरिका भी इससे अलग नहीं हैं । <end>', -7.655535)


In [46]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[900])
print("="*50)
print("ACTUAL OUTPUT ===> ",df_test.dec_output.values[900])
print("="*50)
print("BEAM SEARCH OUTPUT ,  SCORE")
bm = (beam_search(df_test.enc_input.values[900],model,3))
for i in bm:
    print(i)

INPUT SENTENCE ===>  इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक दवाई बनाई हैं ।
ACTUAL OUTPUT ===>  इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक दवाई बनाई है ।  <end>
BEAM SEARCH OUTPUT ,  SCORE
('इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक दवाई बनाई है । <end>', -0.002123482)
('इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक खुल बनाई है । <end>', -9.93136)
('इसलिए वैज्ञानिकों ने लाइकोपीन तबाही एक दवाई बनाई है । <end>', -10.147559)


In [72]:
# On test dataset
df_test1= pd.read_csv("DATA/etoori_test.csv")
df_test1.columns = ["enc_input","dec_input"] 
df_test1["dec_output"] = df_test1.dec_input
print(df_test1.shape)
df_test1

(30000, 3)


,enc_input,dec_input,dec_output
0,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...
1,यह मन को काबू में करने वाली मुद्रा हैं इसीलिए ...,यह मन को काबू में करने वाली मुद्रा है इसीलिए इ...,यह मन को काबू में करने वाली मुद्रा है इसीलिए इ...
2,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ...,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ प...,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ प...
3,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...
4,उनकी वो वाली बात भी अनिश्चित रहती हैं ।,उनकी वो वाली बात भी अनिश्चित रहती है ।,उनकी वो वाली बात भी अनिश्चित रहती है ।
...,...,...,...
29995,पालिश घर्षक मशीनें करती हैंं ।,पालिश घर्षक मशीनें करती हैं ।,पालिश घर्षक मशीनें करती हैं ।
29996,पहले पाकिस्तान ने हमले में भारत का हाथ बताया औ...,पहले पाकिस्तान ने हमले में भारत का हाथ बताया औ...,पहले पाकिस्तान ने हमले में भारत का हाथ बताया औ...
29997,सारे का सारा दर्द उन्हों अप ह्रदय में ही समेट ...,सारे का सारा दर्द उन्होंने अपने ह्रदय में ही स...,सारे का सारा दर्द उन्होंने अपने ह्रदय में ही स...
29998,विल्सन ने लिखे पत्र में रिपब्लिकन सांसदों से भ...,विल्सन ने लिखे पत्र में रिपब्लिकन सांसदों से भ...,विल्सन ने लिखे पत्र में रिपब्लिकन सांसदों से भ...


In [73]:
df_test1 = df_test1.head(10000)
df_test1["dec_input"]= "<start> " + df_test1["dec_input"]
df_test1["dec_output"] =  df_test1["dec_output"] + " <end>" 

In [ ]:
#test_ = df_test1.head(10)

In [74]:
import nltk.translate.bleu_score as bleu
BLEU_val_emb = []
test_data = df_test1
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = predict(str(i.enc_input),model)[0].split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        #print("BELU Score",b)
        BLEU_val_emb.append(b)
    except:
        continue
print("BELU Score = ",np.mean(BLEU_val_emb))

(10000, 3)


10000it [23:54,  6.97it/s]

BELU Score =  0.7663164673918077


In [ ]:
#Testing on 30k data points

In [77]:
# On test dataset
df_test1= pd.read_csv("DATA/etoori_test.csv")
df_test1.columns = ["enc_input","dec_input"] 
df_test1["dec_output"] = df_test1.dec_input
print(df_test1.shape)
df_test1["dec_input"]= "<start> " + df_test1["dec_input"]
df_test1["dec_output"] =  df_test1["dec_output"] + " <end>" 

(30000, 3)


In [ ]:
BLEU_val_emb = []
test_data = df_test1
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        #pred = predict(str(i.enc_input),model).split()
        pred = predict(str(i.enc_input),model)[0].split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_val_emb.append(b)
    except:
        continue



(30000, 3)


8155it [18:58,  7.15it/s]

In [ ]:
print("BELU Score for 30k data points= ",np.mean(BLEU_val_emb))

In [ ]:
print("BELU Score= ",np.mean(BLEU_val_emb))

In [57]:
print("INPUT SENTENCE ===> ",test_data.enc_input.values[10])
print("PREDICTED SENTENCE ===> ",predict(test_data.enc_input.values[10],model)[0])
print("ACTUAL SENTENCE ===> ",test_data.dec_output.values[10])

INPUT SENTENCE ===>  राजनैतिक गलियारों में इन दिनों यह चर्चा गर्म है कि सांगार क्षेत्र से पिछले पांच सालों से विधायक की दावेदारी का खुद का मजबूत दावेदार समझ वाले एक ता की दावेदारी बेहद फीकी है ।
PREDICTED SENTENCE ===>  सालों की दावेदारी का खुद का मजबूत दावेदार समझ वाले एक नेता की दावेदारी बेहद फीकी हैं । । <end>
ACTUAL SENTENCE ===>  राजनैतिक गलियारों में इन दिनों यह चर्चा गर्म है कि सांगानेर क्षेत्र से पिछले पांच सालों से विधायक की दावेदारी का खुद का मजबूत दावेदार समझने वाले एक नेता की दावेदारी बेहद फीकी है ।  <end>


In [58]:
print("INPUT SENTENCE ===> ",test_data.enc_input.values[1])
print("PREDICTED SENTENCE ===> ",predict(test_data.enc_input.values[1],model)[0])
print("ACTUAL SENTENCE ===> ",test_data.dec_output.values[1])

INPUT SENTENCE ===>  यह मन को काबू में करने वाली मुद्रा हैं इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैंं ।
PREDICTED SENTENCE ===>  यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं । <end>
ACTUAL SENTENCE ===>  यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं ।  <end>


In [59]:
print("INPUT SENTENCE ===> ",test_data.enc_input.values[100])
print("PREDICTED SENTENCE ===> ",predict(test_data.enc_input.values[100],model)[0])
print("ACTUAL SENTENCE ===> ",test_data.dec_output.values[100])

INPUT SENTENCE ===>  शोधकर्ताओं परिकल्पना हैं कि इस
PREDICTED SENTENCE ===>  शोधकर्ताओं ने परिकल्पना है कि इस <end>
ACTUAL SENTENCE ===>  शोधकर्ताओं परिकल्पना है कि इस <end>


In [60]:
print("INPUT SENTENCE ===> ",test_data.enc_input.values[200])
print("PREDICTED SENTENCE ===> ",predict(test_data.enc_input.values[200],model)[0])
print("ACTUAL SENTENCE ===> ",test_data.dec_output.values[200])

INPUT SENTENCE ===>  दूसरे एफडीआया संसद में भी पारित हो गया है ।
PREDICTED SENTENCE ===>  दूसरे एफडीआई संसद में भी पारित हो गया है । <end>
ACTUAL SENTENCE ===>  दूसरे एफडीआई संसद में भी पारित हो गया है ।  <end>


In [62]:
print("INPUT SENTENCE ===> ",test_data.enc_input.values[500])
print("PREDICTED SENTENCE ===> ",predict(test_data.enc_input.values[500],model)[0])
print("ACTUAL SENTENCE ===> ",test_data.dec_output.values[500])

INPUT SENTENCE ===>  क्योंकि एडवीज एंटोनिया अलवीना माइनो उर्फ सोनिया गांधी का धन स्विस बैंक में भरा पड़ा है इसीलिए सोनिया गांधी के गुलाम प्रधानमन्त्री मनमोहन सिंह जर्मनी द्वारा सविस बैंक में काला धन रख वालों की सूची आफर कर के बाबजूद सूची ले से मना कर दिया था ।
PREDICTED SENTENCE ===>  बैंक ने धन रखने वालों की सूची आफर करने के बाबजूद सूची लेने के बाबजूद सूची ले से मना कर
ACTUAL SENTENCE ===>  क्योंकि एडवीज एंटोनिया अलवीना माइनो उर्फ सोनिया गांधी का धन स्विस बैंक में भरा पड़ा है इसीलिए सोनिया गांधी के गुलाम प्रधानमन्त्री मनमोहन सिंह ने जर्मनी द्वारा सविस बैंक में काला धन रखने वालों की सूची आफर करने के बाबजूद सूची लेने से मना कर दिया था ।  <end>


In [64]:
print("INPUT SENTENCE ===> ",test_data.enc_input.values[950])
print("PREDICTED SENTENCE ===> ",predict(test_data.enc_input.values[950],model)[0])
print("ACTUAL SENTENCE ===> ",test_data.dec_output.values[950])

INPUT SENTENCE ===>  इस बात का खुलासा होने के बाद नक्सलियों के स्याह चेहरे का एक और बदनुमा रंग सामने आ गया हैं ।
PREDICTED SENTENCE ===>  बात का खुलासा होने के बाद नक्सलियों के स्याह चेहरे का एक और बदनुमा रंग सामने आ गया है ।
ACTUAL SENTENCE ===>  इस बात का खुलासा होने के बाद नक्सलियों के स्याह चेहरे का एक और बदनुमा रंग सामने आ गया है ।  <end>


In [65]:
print("INPUT SENTENCE ===> ",test_data.enc_input.values[800])
print("PREDICTED SENTENCE ===> ",predict(test_data.enc_input.values[800],model)[0])
print("ACTUAL SENTENCE ===> ",test_data.dec_output.values[800])

INPUT SENTENCE ===>  हां कहीं यह बहुत खुलकर साम आता है और कहीं अंदर ही अंदर पलता रहता है ।
PREDICTED SENTENCE ===>  हां कहीं यह बहुत खुलकर सामने आता है और कहीं अंदर ही अंदर पलता रहता है । <end>
ACTUAL SENTENCE ===>  हां कहीं यह बहुत खुलकर सामने आता है और कहीं अंदर ही अंदर पलता रहता है ।  <end>


In [66]:
test_data.shape

(30000, 3)